In [9]:
import torch
import numpy as np
import pandas as pd
import datetime
from libs.sequences import SeismicSequence
from libs.iris import irisRequests
from importlib import reload  # Python 3.4+
import matplotlib.pyplot as plt

In [2]:
# specify regions limits
regions = {}
regions['greece'] = (30, 45,18, 44)
regions['california'] = (30, 41, -125, -113)
regions['japan'] = (20, 50, 120, 150)
regions['italy'] = (35,46,6, 19)

In [3]:
region = regions['japan']
start_time = datetime.datetime(2012, 1, 1, 0, 0, 0)
end_time =  datetime.datetime(2024, 1, 1, 0, 0, 0)
download_url =irisRequests.url_events_box(start_time, end_time, region[0], region[1], region[2], region[3], minmag=3, magtype="MW")
df = pd.read_csv(download_url, sep="|", comment="#")
df.Time = pd.to_datetime(df.Time, errors='coerce')
df.dropna(axis=0, inplace=True)
df.sort_values(by="Time", inplace=True)
df.reset_index(inplace=True, drop=True)

In [5]:
seq = SeismicSequence.from_pandas_df(df)

In [4]:
n=20
inter_times = np.random.exponential(size=n+1)
feats = np.random.randn(n, 3) 
seq = SeismicSequence(inter_times, features=feats)

In [13]:
seq2 = seq.get_subsequence(2, 10)

In [17]:
SeismicSequence.pack_sequences([seq, seq2])[2]

tensor([20,  8])

In [19]:
seq2